In [202]:
import hashlib

df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
def hash_row(row):
    row_string = str(row['Application_Number'])
    return hashlib.sha256(row_string.encode()).hexdigest()

df_raw['Application_Number_hashed'] = df_raw.apply(hash_row, axis=1)
df_raw.to_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131_withhasedappId.csv', index=False)


Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.


In [35]:
import pandas as pd

final_model_path = '/d/shared/silver_projects_v2/origencepoc/autoloanv4/modeling/model_artifacts/autoloan/model1Ensemble'
scores_df = pd.read_parquet(os.path.join(final_model_path, 'penfed_autoindirectv1_scores.parquet'))
ana_vs40 = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/processed/supporting_artifacts/raw_analysis_data_with _VS40.parquet')
ana_raw = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/processed/supporting_artifacts/analysis_data.parquet')
ana = ana_raw.merge(ana_vs40[['ZEST_KEY', 'EFX_VS40']].set_index('ZEST_KEY'), how='left', left_index=True, right_index=True)

ana = ana.merge(scores_df[['final_model_predictions']], how='left', left_index=True, right_index=True)
# ana.to_parquet('./final_merged_data.parquet')

In [36]:
df_book = ana[ana['flgFunded']==1]
df_unbook = ana[ana['flgFunded']==0]

In [20]:
proxy_loans_df['mob_24_120'].value_counts()

mob_24_120
0.0    102005
1.0      6672
Name: count, dtype: int64

In [146]:
proxy_loans_df = pd.read_parquet('./auto_proxy_targets.parquet')

df_unbook_proxy = df_unbook.drop(columns=['appId', 'flgFunded', 'flgApproved','Funded_Date','appDate','mob_3_30',
 'mob_4_60',
 'mob_5_60',
 'mob_6_60',
 'mob_6_90',
 'mob_12_90',
 'mob_12_120',
 'mob_18_90',
 'mob_18_120',
 'mob_24_90',
 'mob_24_120']).merge(proxy_loans_df.set_index('ZEST_KEY'), left_index = True, 
                                  right_index = True,
                                  how = 'left').reset_index(drop = False)
df_unbook_proxy['abs_appdate_minus_opendate'] = (df_unbook_proxy['Funded_Date'] - df_unbook_proxy['appDate']).abs().dt.days
df_unbook_w_targets = df_unbook_proxy.sort_values(['ZEST_KEY', 'abs_appdate_minus_opendate', 'mob_6_60'],
                                   ascending = [True, True, False]).drop_duplicates(['ZEST_KEY'],keep='first')

In [142]:
proxy_target_all_merge = pd.read_csv('/d/shared/users/yyz/penfed/proxy_target_part1.csv')

In [144]:
df_unbook.shape, proxy_loans_df.shape, df_unbook_proxy.shape,df_unbook_w_targets.shape

((497471, 181), (166029, 219), (518565, 384), (497471, 384))

In [ ]:
df_unbook_w_targets['DATE_REPORTED']

In [60]:
import pandas as pd


In [51]:
df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')



/tmp/ipykernel_220/1885037264.py:1: DtypeWarning: Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')


In [39]:
import numpy as np
df = df_raw.copy()
df = df[df['Application_Status'].isin(['Refer', 'Processing', 'Exception'])==False]
print(df.shape) # 561,137
df = df[df['Max_Credit_Risk_Score']>=610] # 537656
print(df.shape) # 537, 652

df = df[~((df['PenFed_Funded']==1) & (df['Lendpro_Cert_no'].notnull()))] # exclude insured
df = df[~((df['PenFed_Funded']==1) & (df['cancelled_flag']==1))] # exclude cancelled
df = df[~((df['PenFed_Funded']==1) & (df['never_cash_flag']==1))] # exclude never cash the check
df = df[~((df['PenFed_Funded']==1) & (df['fraud_3rd_party']==1))] # exclude 3rd party fraud


df['Decision'] = None
df['Decision'] = np.where(df['PenFed_Funded']==1, 'Funded', df['Decision'])
df['Decision'] = np.where((df['PenFed_Funded']==0) & 
                          ((df['Application_Status']=='Approve')|((df['Application_Status']=='Counter')&(df['Adverse_Action_Reasons'].isnull()))), 'Withdraw', df['Decision'])
df['Decision'] = np.where((df['PenFed_Funded']==0) & 
                          ((df['Application_Status']=='Decline')|((df['Application_Status']=='Counter')&(df['Adverse_Action_Reasons'].notnull()))), 'Decline', df['Decision'])


df['Decision'].value_counts(dropna=False)


(631588, 126)
(606073, 126)


Decision
Withdraw    258114
Decline     197700
Funded      125847
Name: count, dtype: int64

In [156]:
df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
df_raw['Decision'] = None
df_raw['Decision'] = np.where(df_raw['PenFed_Funded']==1, 'Funded', df_raw['Decision'])
df_raw['Decision'] = np.where((df_raw['PenFed_Funded']==0) & 
                          ((df_raw['Application_Status']=='Approve')|((df_raw['Application_Status']=='Counter')&(df_raw['Adverse_Action_Reasons'].isnull()))), 'Withdraw', df_raw['Decision'])
df_raw['Decision'] = np.where((df_raw['PenFed_Funded']==0) & 
                          ((df_raw['Application_Status']=='Decline')|((df_raw['Application_Status']=='Counter')&(df_raw['Adverse_Action_Reasons'].notnull()))), 'Decline', df_raw['Decision'])

df_raw['flgBorrower'] = df_raw['Role_Code'].map({'PRIMARY': 1, 'JOINTOWN': 2})
df_raw['ZEST_KEY'] = df_raw['Application_Number'].astype(str)+'_'+df_raw['flgBorrower'].astype(str)+'_229_22'

Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.


In [256]:
df_book.shape, df_unbook_w_targets.shape

((154338, 181), (497471, 384))

In [157]:
common_cols = [
               'ZEST_KEY', 'Funded_Date','mob_6_60', 'mob_12_90', 'mob_12_120','mob_18_90','mob_18_120','mob_24_120',
               'final_model_predictions', 'EFX_VS40'
              ]

df_all = pd.concat([df_book.reset_index(drop=False)[common_cols], df_unbook_w_targets[common_cols]]).reset_index(drop=False)

drop_common_cols = list(set(df_all.columns)&set(df_raw.columns)-set(['ZEST_KEY', 'Decision']))

df_merged = df_raw.drop(columns=drop_common_cols).merge(df_all, how='left', on=['ZEST_KEY'])


In [150]:
penfed_new = pd.read_csv('./data_3rd_party_refi_addvar_sent_20250613210112.csv')

In [158]:
full_merged = penfed_new.merge(df_merged, how='inner', on=['Application_Number', 'Application_Date', 'Role_Code']).drop(columns=['index'])
full_merged = full_merged.drop_duplicates()
full_merged.to_csv('/d/shared/users/yyz/penfed/data_3rd_party_refi_f_sent_withunfundedtarget_zestscore_vs4_20250618.csv', index=False)

In [81]:
df_raw[df_raw.duplicated(['Application_Number','Application_Date','Role_Code'])][['Application_Number']]

,Application_Number
458991,88800617
459697,88766767
500542,89236327


In [105]:
full_merged.shape

(651816, 142)

In [283]:
df_merged.to_csv('/d/shared/users/yyz/penfed/data_3rd_party_refi_f_sent_withunfundedtarget_zestscore_vs4.csv', index=False)

In [102]:
full_merged['mob_24_120'] = full_merged['mob_24_120'].astype(float)
full_merged['appmonth'] = full_merged['Application_Date'].str[:7]
full_merged.groupby('appmonth')['mob_24_120'].sum()

appmonth
2021-01      79.0
2021-02     108.0
2021-03     149.0
2021-04     189.0
2021-05     213.0
2021-06     346.0
2021-07     400.0
2021-08     460.0
2021-09     518.0
2021-10     646.0
2021-11     859.0
2021-12     928.0
2022-01     982.0
2022-02     935.0
2022-03    1341.0
2022-04    1241.0
2022-05    1268.0
2022-06    1245.0
2022-07     555.0
2022-08     303.0
2022-09     192.0
2022-10     237.0
2022-11     219.0
2022-12     220.0
2023-01     235.0
2023-02     245.0
2023-03     250.0
2023-04      39.0
2023-05      10.0
2023-06       1.0
2023-07       0.0
2023-08       0.0
2023-09       0.0
2023-10       0.0
2023-11       0.0
2023-12       0.0
2024-01       0.0
2024-02       0.0
2024-03       0.0
2024-04       0.0
2024-05       0.0
2024-06       0.0
2024-07       0.0
2024-08       0.0
2024-09       0.0
2024-10       0.0
2024-11       0.0
2024-12       0.0
2025-01       0.0
2025-02       0.0
2025-03       0.0
2025-04       0.0
Name: mob_24_120, dtype: float64

In [110]:
import pandas as pd
import ztarget
import model_engine
import json

ztarget.__version__, model_engine.__version__

('1.2.1', 'v1.12.4')

In [62]:
# Zest analysis data 
ana = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/processed/supporting_artifacts/analysis_data.parquet')
ana.shape

(651809, 179)

In [63]:
# PenFed Sent Data
df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
df_raw.shape

/tmp/ipykernel_163/4184390068.py:2: DtypeWarning: Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')


(651816, 126)

# Unfunded Proxy Target

In [111]:
from ztarget import EQTargetGenerator, TUTargetGenerator, EXPTargetGenerator
from ztarget.target_generator import final_target
from model_engine.assets.utils import load_asset
from itertools import product

pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
path = 's3://power-client-data-staging/CLIENT/PARSED/DATA/BUREAU%3Dequifax/FORMAT%3Dcms_6/TABLE%3Dtrade/PULL_NAME%3D20250529_penfed/ARCHIVE_DATE%3D2025-04-30/'.replace('%3D', '=')
perf_trade = pd.read_parquet(path)

In [ ]:
unfunded_ana = ana[ana.flgFunded == 0]
unfunded_ana.shape

In [ ]:
unfunded_trade = perf_trade[perf_trade.ZEST_KEY.isin(unfunded_ana.index)]
unfunded_trade.ZEST_KEY.nunique()

## Auto Part 1 – Auto Loan Proxy

Return a record for all Auto Loans (exclude leases) opened within -1/+3 months from app month

Fields returned for each proxy trade:
- MOB6 Ever 60+DPD/BNK/Repossession/Charged-off
- MOB12 Ever 90+DPD/BNK/Repossession/Charged-off (General Bad PenFed used for model development)
- MOB12 Ever 120+DPD/Repossession/Charged-off (Simulate PenFed current Charge off Policy. Does not include BNK)
- MOB18 Ever 90+DPD/BNK/Repossession/Charged-off
- MOB18 Ever 120+DPD/Repossession/Charged-off
- MOB24 Ever 120+DPD/Repossession/Charged-off
- Payment history grid (pulled from current archive)
	
If there are multiple proxy trades for an individual application, we want to keep all of them as reference.


In [112]:
eq_trade_json = load_asset('equifax/cms_6/trade.json')
eq_trade_json_nobnk = load_asset('equifax/cms_6/trade.json') # remove 7 (Included in Chapter 13) from Charge-off
eq_trade_json_nobnk['info']['payment_patterns'] = {'patterns': ['PAYMENT_HISTORY_1_24',
  'PAYMENT_HISTORY_25_36',
  'PAYMENT_HISTORY_37_48'],
 'rate': {'paid_as_agreed': ['0', '1'],
  'DQ30': ['2'],
  'DQ60': ['3'],
  'DQ90': ['4'],
  'DQ120': ['5'],
  'CO': ['5', '6', '8', '9', 'G', 'H', 'K', 'L']},
 'trim': 48,
 'placeholder': '/',
 'keep': ['DQ30', 'DQ60', 'DQ90', 'DQ120', 'CO']}


In [113]:
dqs = ["DQ30", "DQ60", "DQ90", "DQ120", "CO"]
time_window = [3, 6, 9, 12, 15, 18, 24, 30, 36]
default_proxy_time_window = [-30, 90]

# exclude 3A: Auto Lease 
# exclude 11: recreational merchandise

proxy_account_type = ['00']
target_generator = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json['info'],
                                     proxy = True)
target_generator.proxy_account_type

target_generator_nobnk = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json_nobnk['info'],
                                     proxy = True)
target_generator.proxy_account_type

['00']

In [ ]:
cols = ['appId', 'appDate', 'flgFunded', 'flgApproved',]
unfunded_trade = unfunded_trade.merge(unfunded_ana[cols], left_on = 'ZEST_KEY', 
                       right_index = True)

In [39]:
proxy_date_df = target_generator.get_dqDates(unfunded_trade)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert LAST_PAYMENT_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
T

In [26]:
proxy_date_df_nobnk = target_generator_nobnk.get_dqDates(unfunded_trade)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert LAST_PAYMENT_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
T

In [30]:
# proxy_date_df.to_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/full_auto_loan_proxy.parquet')

In [40]:
proxy_date_df_nobnk.to_parquet('/d/shared/users/yyz/penfed/full_auto_loan_proxy_nobnk.parquet')
proxy_date_df.to_parquet('/d/shared/users/yyz/penfed/full_auto_loan_proxy.parquet')

In [114]:
proxy_date_df = pd.read_parquet('/d/shared/users/yyz/penfed/full_auto_loan_proxy.parquet')
proxy_date_df_nobnk = pd.read_parquet('/d/shared/users/yyz/penfed/full_auto_loan_proxy_nobnk.parquet')

proxy_date_df.shape

(166029, 106)

In [115]:
with open(f"/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/bureau_config.json", "r") as file: 
    config = json.load(file)
config

{'conf': {'BUREAU': 'equifax',
  'FORMAT': 'cms_6',
  'PULL_DATE': '2025-05-29',
  'PULL_NAME': '20250529_penfed',
  'PERFORMANCE_DATE': '2025-04-30'}}

In [116]:
pull_date = config['conf']['PULL_DATE']
proxy_target = target_generator.generate_target(proxy_date_df, 
                                         target=dqs, 
                                         time_window=time_window, 
                                         query_date= pull_date)


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

In [117]:
proxy_target_nobnk = target_generator_nobnk.generate_target(proxy_date_df_nobnk, 
                                         target=dqs, 
                                         time_window=time_window, 
                                         query_date=pull_date)

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

In [209]:
proxy_target.shape, proxy_target_nobnk.shape

((166029, 408), (166029, 408))

In [119]:
bnk_column_mapping = \
    {'proxy_DQ60_m6': 'mob_6_60',
     'proxy_DQ60_m12': 'mob_12_60',
      'proxy_DQ90_m12': 'mob_12_90',
      'proxy_DQ90_m18': 'mob_18_90',
     'proxy_DQ120_m12': 'mob_12_120',
    'proxy_DQ120_m18': 'mob_18_120',
    'proxy_DQ120_m24': 'mob_24_120',
      'proxy_DQ60_m6_status': 'mob_6_60_status',
      'proxy_DQ60_m12_status': 'mob_12_60_status',
      'proxy_DQ90_m12_status': 'mob_12_90_status',
      'proxy_DQ90_m18_status': 'mob_18_90_status',
     'proxy_DQ120_m12_status': 'mob_12_120_status',
    'proxy_DQ120_m18_status': 'mob_18_120_status',
    'proxy_DQ120_m24_status': 'mob_24_120_status',
}

nobnk_column_mapping = {
    'proxy_DQ120_m12': 'mob_12_120_nobnk',
    'proxy_DQ120_m18': 'mob_18_120_nobnk',
    'proxy_DQ120_m24': 'mob_24_120_nobnk',
    'proxy_DQ120_m12_status': 'mob_12_120_nobnk_status',
    'proxy_DQ120_m18_status': 'mob_18_120_nobnk_status',
    'proxy_DQ120_m24_status': 'mob_24_120_nobnk_status',
}

date_source_cols = [c for c in proxy_target.columns if c.endswith('date_source')]

proxy_target_rename = proxy_target.rename(columns=bnk_column_mapping)
proxy_target_nobnk_rename = proxy_target_nobnk.rename(columns=nobnk_column_mapping)

proxy_target_nobnk_keep_cols = list(nobnk_column_mapping.values())

proxy_target_all = pd.concat([proxy_target_rename, \
                                       proxy_target_nobnk_rename[proxy_target_nobnk_keep_cols]
                                       ], axis=1)

keep_cols = ['ZEST_KEY','DATE_OPENED','DATE_REPORTED','ACCOUNT_TYPE','HIGH_CREDIT','SCHEDULED_PAYMENT_AMOUNT','TERMS_DURATION','TERMS_FREQUENCY',\
             'PAYMENT_HISTORY_1_24','PAYMENT_HISTORY_25_36','PAYMENT_HISTORY_37_48'] + \
            list(bnk_column_mapping.values())+list(nobnk_column_mapping.values())
proxy_target_all_subset = proxy_target_all[keep_cols]

df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131_withhasedappId.csv')

df_raw['flgBorrower'] = df_raw['Role_Code'].map({'PRIMARY': 1, 'JOINTOWN': 2})
df_raw['ZEST_KEY'] = df_raw['Application_Number'].astype(str)+'_'+df_raw['flgBorrower'].astype(str)+'_229_22'
keep_cols = ['ZEST_KEY','Application_Number','Application_Number_hashed','Application_Date','Role_Code','Seller_Name','Product','Application_Status','PenFed_Funded']
df_raw = df_raw[keep_cols]
proxy_target_all_merge = df_raw.merge(proxy_target_all_subset, how='inner', on=['ZEST_KEY'])\
    .drop(columns=['ZEST_KEY','mob_12_120','mob_18_120','mob_24_120','mob_12_120_status','mob_18_120_status','mob_24_120_status'])

Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.


In [120]:
proxy_target_all_merge.shape, proxy_target_all_subset.shape

((166029, 32), (166029, 31))

In [242]:
proxy_target_all_merge.shape, proxy_target_all_subset.shape

((166029, 31), (166029, 31))

In [122]:
proxy_target_all_merge.to_csv('/d/shared/users/yyz/penfed/proxy_target_part1.csv', index=False)

In [121]:
proxy_target_all_merge['appmonth'] = proxy_target_all_merge['Application_Date'].str[:7]
proxy_target_all_merge.groupby('appmonth')['mob_24_120_nobnk'].sum()

appmonth
2021-01    35.0000
2021-02    54.0000
2021-03    74.0000
2021-04   106.0000
2021-05   118.0000
2021-06   193.0000
2021-07   174.0000
2021-08   228.0000
2021-09   227.0000
2021-10   334.0000
2021-11   377.0000
2021-12   411.0000
2022-01   431.0000
2022-02   411.0000
2022-03   585.0000
2022-04   512.0000
2022-05   573.0000
2022-06   535.0000
2022-07   327.0000
2022-08   154.0000
2022-09   110.0000
2022-10   132.0000
2022-11   109.0000
2022-12   101.0000
2023-01   109.0000
2023-02   115.0000
2023-03    80.0000
2023-04    46.0000
2023-05    10.0000
2023-06     1.0000
2023-07     0.0000
2023-08     0.0000
2023-09     0.0000
2023-10     0.0000
2023-11     0.0000
2023-12     0.0000
2024-01     0.0000
2024-02     0.0000
2024-03     0.0000
2024-04     0.0000
2024-05     0.0000
2024-06     0.0000
2024-07     0.0000
2024-08     0.0000
2024-09     0.0000
2024-10     0.0000
2024-11     0.0000
2024-12     0.0000
2025-01     0.0000
2025-02     0.0000
2025-03     0.0000
Name: mob_24_120_nobnk

In [233]:
proxy_target_all_merge['appmonth'] = proxy_target_all_merge['Application_Date'].str[:7]
proxy_target_all_merge.groupby('appmonth')['mob_24_120_nobnk'].sum()

appmonth
2021-01    35.0000
2021-02    54.0000
2021-03    74.0000
2021-04   106.0000
2021-05   118.0000
2021-06   193.0000
2021-07   174.0000
2021-08   228.0000
2021-09   227.0000
2021-10   334.0000
2021-11   377.0000
2021-12   411.0000
2022-01   431.0000
2022-02   411.0000
2022-03   585.0000
2022-04   512.0000
2022-05   573.0000
2022-06   535.0000
2022-07   327.0000
2022-08   154.0000
2022-09   110.0000
2022-10   132.0000
2022-11   109.0000
2022-12   101.0000
2023-01   109.0000
2023-02   115.0000
2023-03    80.0000
2023-04    46.0000
2023-05    10.0000
2023-06     1.0000
2023-07     0.0000
2023-08     0.0000
2023-09     0.0000
2023-10     0.0000
2023-11     0.0000
2023-12     0.0000
2024-01     0.0000
2024-02     0.0000
2024-03     0.0000
2024-04     0.0000
2024-05     0.0000
2024-06     0.0000
2024-07     0.0000
2024-08     0.0000
2024-09     0.0000
2024-10     0.0000
2024-11     0.0000
2024-12     0.0000
2025-01     0.0000
2025-02     0.0000
2025-03     0.0000
Name: mob_24_120_nobnk

## Auto Part 2 – Waterfall Proxy – Only Proceed to this step if no data is found in Part 1 

Return one record max per applicant (the first that is found in the waterfall)

1. Non-Mortgage Secured Loan opened -1 to 1 month from application month
2. Non-Mortgage Secured Loan opened -1 to 3 month from application month
3. Non-Mortgage Secured Loan opened -3 to 3 month from application month 
4. Existing Auto Loan (must be paid as agreed at app month)
5. Existing Personal Loan (must be paid as agreed at app month)
6. Any Open Loan (must be paid as agreed at app month)

Duplicate treatment logic (for Part 2 only)
Choose worst performing trade (60+DPD M12), next choose trade opened closest to app month 
Else random select from duplicate trades

Fields returned for the single proxy, if found:
Waterfall # that generated the trade 
- Date Opened, Loan Amount, Terms, Estimated APR
- MOB6 Ever 60+DPD/BNK/Repossession/Charged-off
- MOB12 Ever 90+DPD/BNK/Repossession/Charged-off (General Bad PenFed used for model development)
- MOB12 Ever 120+DPD/Repossession/Charged-off (Simulate PenFed current Charge off Policy. Does not include BNK)
- MOB18 Ever 90+DPD/BNK/Repossession/Charged-off
- MOB18 Ever 120+DPD/Repossession/Charged-off
- MOB24 Ever 120+DPD/Repossession/Charged-off
- Payment history grid (pulled from current archive)


In [ ]:
import numpy as np
def udpate_date_formats(data, dates): 
    for date_col, date_format in dates.items():
        if np.issubdtype(data[date_col].dtype, np.datetime64):
            print(f'{date_col} is already of datetime format! Will skip the converting process')
            continue
        data[date_col] = data[date_col].astype('str')
        for date_col, date_format in dates.items(): 
            if date_format == '%m%d%Y':
                idx = data[date_col].str[2:4] == '00'
                data.loc[idx, date_col] = data.loc[idx, date_col].str[0:2] + '01' + data.loc[idx, date_col].str[4:]
                data[date_col] = data[date_col].replace('        ', np.nan)
            try:
                data[date_col] = pd.to_datetime(data[date_col], format=date_format, errors='raise')
            except:
                print(f'Warning! Cannot covert {date_col} to datetime using {date_format} format! Please check the asset dates format!' )
                print('Trying to convert datetime using inferred datetime format...')
                data[date_col] = pd.to_datetime(data[date_col],format=date_format, errors='coerce')
    return data         

In [ ]:
proxy_date_df = pd.read_parquet('/d/shared/users/yyz/penfed/full_auto_loan_proxy.parquet')

### A. Non-Mortgage Secured Loans

In [ ]:
unfunded_ana_v1 = unfunded_ana[~unfunded_ana.index.isin(proxy_date_df.ZEST_KEY)]

In [ ]:
# 02: secured
# 22: Secured By Household Goods
# 23: Secured By Household Goods/Collateral

secured_account_type = ['02', '22', '23']

cols = ['ZEST_KEY', 'appDate']
unfunded_trade_v1 = unfunded_trade[unfunded_trade.ZEST_KEY.isin(unfunded_ana_v1.index)]
unfunded_trade_v1 = unfunded_trade_v1.merge(unfunded_ana_v1.reset_index(drop=False)[cols], how='left', on=['ZEST_KEY'])
dates = {
    'DATE_REPORTED': '%m%d%Y',
    'DATE_OPENED': '%m%d%Y',
}
unfunded_trade_v1 = udpate_date_formats(unfunded_trade_v1, dates)

unfunded_trade_v1_filter = unfunded_trade_v1[(unfunded_trade_v1.ACCOUNT_TYPE.isin(secured_account_type))&(unfunded_trade_v1.PORTFOLIO_TYPE != 'M')]
unfunded_trade_v1_filter['days_diff'] = (unfunded_trade_v1_filter['DATE_OPENED'] - unfunded_trade_v1_filter['appDate']).dt.days
unfunded_trade_v1_filter = unfunded_trade_v1_filter[(unfunded_trade_v1_filter['days_diff']>=-90)&(unfunded_trade_v1_filter['days_diff']<=90)]


def classify_loan_timing(days_diff):
    if -30 <= days_diff <= 30:
        return 1
    elif -30 <= days_diff <= 90:
        return 2
    elif -90 <= days_diff <= 90:
        return 3
    else:
        return np.nan  # or 0 or any other indicator for not falling into these windows

unfunded_trade_v1_filter['Waterfall_Number'] = unfunded_trade_v1_filter['days_diff'].apply(classify_loan_timing)

min_waterfall = unfunded_trade_v1_filter.groupby('ZEST_KEY')['Waterfall_Number'].transform('min')

# Keep only rows where the waterfall_number is the minimum for that ZEST_KEY
nonmortagesecure_loan = unfunded_trade_v1_filter[unfunded_trade_v1_filter['Waterfall_Number'] == min_waterfall].copy()

In [100]:
unfunded_trade_v1.shape, unfunded_trade_v1_filter.shape, unfunded_trade_v1_filter.ZEST_KEY.nunique(), nonmortagesecure_loan.shape

((7782099, 63), (10367, 65), 8880, (9347, 65))

In [101]:
nonmortagesecure_loan['Waterfall_Number'].value_counts(dropna = False)

Waterfall_Number
1    4087
2    2802
3    2458
Name: count, dtype: int64

### B.Existing Loans

In [84]:
unfunded_ana_v1.shape, unfunded_ana_v2.shape

((352536, 179), (343656, 179))

In [66]:
unfunded_ana_v2 = unfunded_ana_v1[~unfunded_ana_v1.index.isin(nonmortagesecure_loan.ZEST_KEY)]
unfunded_trade_v2 = unfunded_trade[unfunded_trade.ZEST_KEY.isin(unfunded_ana_v2.index)]
unfunded_trade_v2 = unfunded_trade_v2.merge(unfunded_ana_v2.reset_index(drop=False)[cols], how='left', on=['ZEST_KEY'])

dates = {
    'DATE_REPORTED': '%m%d%Y',
    'DATE_OPENED': '%m%d%Y',
}
unfunded_trade_v2 = udpate_date_formats(unfunded_trade_v2, dates)
unfunded_trade_v2['payment_pattern'] = unfunded_trade_v2['PAYMENT_HISTORY_1_24'] + unfunded_trade_v2['PAYMENT_HISTORY_25_36'] + unfunded_trade_v2['PAYMENT_HISTORY_37_48']
unfunded_trade_v2['payment_pattern'] = unfunded_trade_v2['payment_pattern'].fillna('')
unfunded_trade_v2['payment_pattern'] = unfunded_trade_v2['payment_pattern'].str.replace('/', '')

# keep open loans
unfunded_trade_v2_filter = unfunded_trade_v2[unfunded_trade_v2['CLOSED_DATE'].notna()]
unfunded_trade_v2.shape, unfunded_trade_v2_filter.shape

DATE_OPENED is already of datetime format! Will skip the converting process


((7507616, 64), (4034304, 64))

In [67]:
# Precompute periods
unfunded_trade_v2_filter['app_month'] = pd.to_datetime(unfunded_trade_v2_filter['appDate']).dt.to_period('M')
unfunded_trade_v2_filter['report_month'] = pd.to_datetime(unfunded_trade_v2_filter['DATE_REPORTED']).dt.to_period('M')

# Compute month offset (how many months between report and app date)
month_offset = (unfunded_trade_v2_filter['report_month'] - unfunded_trade_v2_filter['app_month']).apply(lambda x: x.n)

# If payment_pattern is a string, we can extract using string indexing
# Invalid indexes will be caught later
pattern_array = unfunded_trade_v2_filter['payment_pattern'].astype(str).values
offset_array = month_offset.values

# Define fast numpy function to extract the code
def get_char(patterns, offsets):
    result = np.full(len(patterns), None, dtype=object)
    for i in range(len(patterns)):
        offset = offsets[i]
        if 0 <= offset < len(patterns[i]):
            result[i] = patterns[i][offset]
    return result

# Apply it
unfunded_trade_v2_filter['status_at_appmonth'] = get_char(pattern_array, offset_array)

In [68]:
autoloan_account_types = {
        "3A": "Auto lease",
        "00": "Auto loan",
        "11": "Recreational merchandise loan"
    }
personalloan_account_types = {
        "01": "Unsecured loan",
        "02": "Secured loan",
        "03": "Partially secured loan",
        "06": "Installment sales contract",
        "0A": "Time-share loan",
        "0F": "Construction loan",
        "13": "Lease",
        "15": "Check, credit or line of credit",
        "20": "Note loan",
        "29": "Rental agreement",
        "47": "Credit line secured revolving terms",
        "4D": "Telecommunications/Cellular",
        "70": "Government overpayment",
        "7B": "Agriculture",
        "92": "Utility company",
        "78": "Installment loan",
        "91": "Debt consolidation "
    }

unfunded_trade_v2_paidasagreed = unfunded_trade_v2_filter[unfunded_trade_v2_filter['status_at_appmonth'].isin(["0","1","E"])]
unfunded_trade_v2_paidasagreed['Waterfall_Number'] = np.where(
    unfunded_trade_v2_paidasagreed['ACCOUNT_TYPE'].isin(autoloan_account_types),
    4,
    np.where(
        unfunded_trade_v2_paidasagreed['ACCOUNT_TYPE'].isin(personalloan_account_types),
        5,
        6
    )
)

min_waterfall = unfunded_trade_v2_paidasagreed.groupby('ZEST_KEY')['Waterfall_Number'].transform('min')
# Keep only rows where the waterfall_number is the minimum for that ZEST_KEY
existing_trade = unfunded_trade_v2_paidasagreed[unfunded_trade_v2_paidasagreed['Waterfall_Number'] == min_waterfall].copy()

In [69]:
existing_trade['Waterfall_Number'].value_counts(dropna = False)


Waterfall_Number
4    191569
6    187168
5     27685
Name: count, dtype: int64

In [102]:
print(nonmortagesecure_loan.shape, existing_trade.shape)
common_cols = list(set(nonmortagesecure_loan.columns) & set(existing_trade.columns))
proxy_trade_waterfall = pd.concat([nonmortagesecure_loan, existing_trade])[common_cols].reset_index(drop=True)
print(proxy_trade_waterfall.shape)


(9347, 65) (406422, 68)
(415769, 64)


In [103]:
default_proxy_time_window = [-36500, 36500]
proxy_account_type = list(set(proxy_trade_waterfall['ACCOUNT_TYPE']))
target_generator = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json['info'],
                                     proxy = True)
target_generator_nobnk = EQTargetGenerator(join_key='ZEST_KEY', app_id='appId', app_date='appDate', 
                                      proxy_time_window = default_proxy_time_window, 
                                      proxy_account_type = proxy_account_type,
                                      **eq_trade_json_nobnk['info'],
                                     proxy = True)

In [104]:
proxy_date_df_waterfall = target_generator.get_dqDates(proxy_trade_waterfall)
proxy_date_df_waterfall_nobnk = target_generator_nobnk.get_dqDates(proxy_trade_waterfall)

--- Transforming date formats ---
Warning! Cannot covert CLOSED_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_BEFORE_HISTORY to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert LAST_PAYMENT_DATE to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DFD_DLA to datetime using %m%d%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert PREVIOUS_HIGH_DATE_3 to datetime using %m%Y format! Please check the asset dates format!
Trying to convert datetime using inferred datetime format...
Warning! Cannot covert DMD_REPORTED to datetime using %m%d%Y format! Please check the asset dates format!
T

In [96]:
proxy_date_df_waterfall.shape, proxy_date_df_waterfall_nobnk.shape

((415728, 103), (415728, 103))

In [152]:
proxy_target_waterfall = target_generator.generate_target(proxy_date_df_waterfall, 
                                         target=dqs, 
                                         time_window=time_window, 
                                         query_date= pull_date)

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

In [153]:
proxy_target_waterfall_nobnk = target_generator_nobnk.generate_target(proxy_date_df_waterfall_nobnk, 
                                         target=dqs, 
                                         time_window=time_window, 
                                         query_date= pull_date)

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

In [154]:
proxy_target_waterfall.shape, proxy_target_waterfall_nobnk.shape

((415728, 405), (415728, 405))

In [155]:
bnk_column_mapping = \
    {'proxy_DQ60_m6': 'mob_6_60',
     'proxy_DQ60_m12': 'mob_12_60',
      'proxy_DQ90_m12': 'mob_12_90',
      'proxy_DQ90_m18': 'mob_18_90',
     'proxy_DQ120_m12': 'mob_12_120',
    'proxy_DQ120_m18': 'mob_18_120',
    'proxy_DQ120_m24': 'mob_24_120',
      'proxy_DQ60_m6_status': 'mob_6_60_status',
      'proxy_DQ60_m12_status': 'mob_12_60_status',
      'proxy_DQ90_m12_status': 'mob_12_90_status',
      'proxy_DQ90_m18_status': 'mob_18_90_status',
     'proxy_DQ120_m12_status': 'mob_12_120_status',
    'proxy_DQ120_m18_status': 'mob_18_120_status',
    'proxy_DQ120_m24_status': 'mob_24_120_status',
}

nobnk_column_mapping = {
    'proxy_DQ120_m12': 'mob_12_120_nobnk',
    'proxy_DQ120_m18': 'mob_18_120_nobnk',
    'proxy_DQ120_m24': 'mob_24_120_nobnk',
    'proxy_DQ120_m12_status': 'mob_12_120_nobnk_status',
    'proxy_DQ120_m18_status': 'mob_18_120_nobnk_status',
    'proxy_DQ120_m24_status': 'mob_24_120_nobnk_status',
}

date_source_cols = [c for c in proxy_target_waterfall.columns if c.endswith('date_source')]

proxy_target_waterfall_rename = proxy_target_waterfall.rename(columns=bnk_column_mapping)
proxy_target_waterfall_nobnk_rename = proxy_target_waterfall_nobnk.rename(columns=nobnk_column_mapping)

proxy_target_waterfall_nobnk_keep_cols = list(nobnk_column_mapping.values())

proxy_target_waterfall_all = pd.concat([proxy_target_waterfall_rename, \
                                       proxy_target_waterfall_nobnk_rename[proxy_target_waterfall_nobnk_keep_cols]
                                       ], axis=1)
proxy_target_waterfall_all['abs_days_diff'] = abs((proxy_target_waterfall_all['DATE_OPENED'] - proxy_target_waterfall_all['appDate']).dt.days)

In [120]:
proxy_target_waterfall_all.shape, proxy_target_waterfall.shape, proxy_target_waterfall_nobnk.shape

((415728, 355), (415728, 405), (415728, 405))

In [3]:
proxy_target_waterfall_all = pd.read_parquet('/d/shared/users/yyz/penfed/proxy_target_waterfall_all.parquet')

In [4]:
# Choose worst performing trade (60+DPD M12), next choose trade opened closest to app month
# Else random select from duplicate trades
proxy_target_waterfall_all_dedup = proxy_target_waterfall_all.sort_values(by=['ZEST_KEY','mob_12_60', 'abs_days_diff'], ascending=[True, False, True])\
    .drop_duplicates(subset=['ZEST_KEY'], keep='first')

In [158]:
proxy_target_waterfall_all.to_parquet('/d/shared/users/yyz/penfed/proxy_target_waterfall_all.parquet')

In [136]:
proxy_target_waterfall_all_dedup.shape, proxy_target_waterfall_all_dedup['ZEST_KEY'].nunique()

((275340, 356), 275340)

In [7]:
keep_cols = ['ZEST_KEY','Waterfall_Number','DATE_OPENED','DATE_REPORTED','ACCOUNT_TYPE','HIGH_CREDIT','SCHEDULED_PAYMENT_AMOUNT','TERMS_DURATION','TERMS_FREQUENCY',\
             'PAYMENT_HISTORY_1_24','PAYMENT_HISTORY_25_36','PAYMENT_HISTORY_37_48'] + \
            list(bnk_column_mapping.values())+list(nobnk_column_mapping.values())
proxy_target_waterfall_all_dedup_subset = proxy_target_waterfall_all_dedup[keep_cols]

df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131_withhasedappId.csv')

df_raw['flgBorrower'] = df_raw['Role_Code'].map({'PRIMARY': 1, 'JOINTOWN': 2})
df_raw['ZEST_KEY'] = df_raw['Application_Number'].astype(str)+'_'+df_raw['flgBorrower'].astype(str)+'_229_22'
keep_cols = ['ZEST_KEY','Application_Number_hashed','Application_Date','Role_Code','Seller_Name','Product','Application_Status','PenFed_Funded']
df_raw = df_raw[keep_cols]
proxy_target_waterfall_all_dedup_merge = df_raw.merge(proxy_target_waterfall_all_dedup_subset, how='inner', on=['ZEST_KEY'])\
    .drop(columns=['ZEST_KEY','mob_12_120','mob_18_120','mob_24_120','mob_12_120_status','mob_18_120_status','mob_24_120_status'])

Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.


In [8]:
proxy_target_waterfall_all_dedup_merge.shape

(275340, 32)

In [240]:
proxy_target_waterfall_all_dedup_merge.to_csv('/d/shared/users/yyz/penfed/proxy_target_waterfall_part2.csv', index=False)

In [12]:
proxy_target_waterfall_all_dedup_merge = pd.read_csv('/d/shared/users/yyz/penfed/proxy_target_waterfall_part2.csv')

In [9]:
trade = proxy_target_waterfall_all_dedup_merge.copy()

import numpy_financial as npf
trade['TERMS_DURATION'] = trade['TERMS_DURATION'].astype(float)
trade['SCHEDULED_PAYMENT_AMOUNT'] = trade['SCHEDULED_PAYMENT_AMOUNT'].astype(float)
trade['HIGH_CREDIT'] = trade['HIGH_CREDIT'].astype(float)


# generate a numeric value for the term frequency; assume monthy if missing
trade['TERMS_FREQUENCY'] = trade['TERMS_FREQUENCY'].fillna('M')
freq_map = {'B':26,'E':24,'L':6,'M':12,'P':1,'Q':4,'S':2,'T':3,'W':52,'Y':1}
trade['TERMS_FREQUENCY_numeric'] = trade['TERMS_FREQUENCY'].map(freq_map)

# filter out invalid rows
## drop rows where bureau loan terms are missing or 0
trade = trade.dropna(subset=['TERMS_DURATION', 'TERMS_FREQUENCY', 'HIGH_CREDIT', 'SCHEDULED_PAYMENT_AMOUNT'])
trade = trade[(trade[['TERMS_DURATION', 'SCHEDULED_PAYMENT_AMOUNT', 'HIGH_CREDIT']] > 0).any(axis=1)]

## drop rows if the full scheduled payment amount is higher than 3 times the loan amount (entries could be a typo and calculated APRs would be too high)
mask = (trade['TERMS_DURATION'] * trade['SCHEDULED_PAYMENT_AMOUNT'] / trade['HIGH_CREDIT']) < 3
trade = trade[mask]
## drop rows if the full scheduled payment amount is less than or equal to the loan amount (entries could be a typo and calculated APRs would be too low)
mask2 = (trade['TERMS_DURATION'] * trade['SCHEDULED_PAYMENT_AMOUNT'] / trade['HIGH_CREDIT']) >= 1
trade = trade[mask2]

# calculate the APR
trade['Estimated_APR'] = npf.rate(
    trade['TERMS_DURATION'],
    -trade['SCHEDULED_PAYMENT_AMOUNT'],
    trade['HIGH_CREDIT'],
    0,
    maxiter=100
    ) * trade['TERMS_FREQUENCY_numeric']

In [24]:
proxy_target_waterfall_all_dedup_merge_apr = proxy_target_waterfall_all_dedup_merge.merge(trade[['Application_Number_hashed', 'Role_Code', 'Estimated_APR']], \
                                             how='left', on=['Application_Number_hashed', 'Role_Code'])
proxy_target_waterfall_all_dedup_merge_apr = proxy_target_waterfall_all_dedup_merge_apr.drop_duplicates()

In [27]:
proxy_target_waterfall_all_dedup_merge_apr.to_csv('/d/shared/users/yyz/penfed/proxy_target_waterfall_part2.csv', index=False)

In [26]:
proxy_target_waterfall_all_dedup_merge.shape, proxy_target_waterfall_all_dedup_merge_apr.shape

((275340, 32), (275339, 33))

In [56]:
275526 - 275340

186

In [17]:
a = proxy_target_waterfall_all_dedup_merge.groupby(['Application_Number_hashed', 'Role_Code'])['Application_Date'].count()
a[a>1]

Application_Number_hashed                                         Role_Code
f0d99b37773cd5b43ccd7e6327446a5dfc807becf04cf48350da2c8d03463144  PRIMARY      2
Name: Application_Date, dtype: int64

In [18]:

tmp = proxy_target_waterfall_all_dedup_merge[proxy_target_waterfall_all_dedup_merge['Application_Number_hashed']=='f0d99b37773cd5b43ccd7e6327446a5dfc807becf04cf48350da2c8d03463144']

In [22]:
tmp.duplicated().sum()

1

In [47]:
proxy_target_waterfall_all_dedup_merge['appmonth'] = proxy_target_waterfall_all_dedup_merge['Application_Date'].str[:7]
proxy_target_waterfall_all_dedup_merge.groupby('appmonth')['mob_24_120_nobnk'].sum()

appmonth
2021-01      7.0
2021-02      7.0
2021-03     23.0
2021-04     26.0
2021-05     31.0
2021-06     43.0
2021-07     75.0
2021-08     61.0
2021-09     70.0
2021-10    147.0
2021-11    141.0
2021-12    131.0
2022-01    169.0
2022-02    153.0
2022-03    228.0
2022-04    246.0
2022-05    284.0
2022-06    294.0
2022-07    139.0
2022-08     58.0
2022-09     61.0
2022-10     43.0
2022-11     54.0
2022-12     53.0
2023-01     45.0
2023-02     43.0
2023-03     53.0
2023-04     58.0
2023-05     63.0
2023-06     27.0
2023-07     21.0
2023-08     21.0
2023-09     16.0
2023-10     11.0
2023-11     11.0
2023-12      6.0
2024-01     15.0
2024-02      7.0
2024-03      9.0
2024-04      8.0
2024-05      3.0
2024-06      0.0
2024-07      1.0
2024-08      0.0
2024-09      2.0
2024-10      2.0
2024-11      1.0
2024-12      0.0
2025-01      0.0
2025-02      1.0
2025-03      1.0
Name: mob_24_120_nobnk, dtype: float64